## BI-Lstm with word embedding and with out padding mask

In [1]:
def read_file(filepath):
    tags=[]
    sentence_tags = []
    sentences = []
    sentence = []
    for line in open(filepath, encoding='utf-8'):
        if line !='\n':
            word, tag = line.strip().split()
            if word.startswith('http://') or word.startswith('https://'):
                word = '<URL>'
            if word.startswith('@'):
                word = '<USR>'
            sentence.append(word)
            sentence_tags.append(tag)
        if line == '\n':
            sentences.append(sentence)
            tags.append(sentence_tags)
            sentence = []
            sentence_tags = []
    return sentences, tags

In [2]:
train_sents, train_tags = read_file('data/train.txt')
val_sents, val_tags = read_file('data/validation.txt')
test_sents, test_tags = read_file('data/test.txt')

In [3]:
for i in range(3):
    for token, tag in zip(train_sents[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [4]:
def build_dict(sents, extra_tokens= None, start_idx=0):
    vocab = set()
    tok2idx = {}
    idx2tok = {}
    for sent in sents:
        vocab.update(sent)
    if extra_tokens:
        vocab.update(extra_tokens)
    
    for i, tok in enumerate(vocab, start_idx):
        idx2tok[i] = tok
    for key, val in idx2tok.items():
        tok2idx[val] = key
    return idx2tok, tok2idx, vocab



In [5]:
sent_idx2tok, sent_tok2idx, vocab = build_dict(train_sents+val_sents, extra_tokens=['<pad>', '<unk>'])

In [6]:
tag_idx2tok, tag_tok2idx, tag_vocab = build_dict(train_tags+val_tags)

In [7]:
sent_tok2idx['<pad>']

19934

In [8]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences

def prep_dataset(sents, tags, sent_dict, tag_dict, vocab, mask=False):
    sample_size = len(sents)
    max_len = max(len(sent) for sent in sents)
    max_len = 41
    sents_seq = [[sent_dict[tok] if tok in vocab else sent_dict['<unk>'] for tok in sent] for sent in sents]
    tags_seq = [[tag_dict[tok] for tok in sent] for sent in tags]
    X = pad_sequences(maxlen=max_len, sequences=sents_seq, value=sent_dict["<pad>"], padding='post')
    if mask:
        y = pad_sequences(maxlen=max_len, sequences=tags_seq, value=tag_dict['<pad>'], padding='post')
    else:
        y = pad_sequences(maxlen=max_len, sequences=tags_seq, value=tag_dict["O"], padding='post')
    return X, y
    

Using TensorFlow backend.


In [9]:
trainX, trainy = prep_dataset(train_sents, train_tags, sent_tok2idx, tag_tok2idx, vocab)

In [10]:
trainX

array([[ 7041,  4471, 15733, ..., 19934, 19934, 19934],
       [ 3719,  8142,  1685, ..., 19934, 19934, 19934],
       [19943,  4975,  4471, ..., 19934, 19934, 19934],
       ...,
       [20394,  9185, 12344, ..., 19934, 19934, 19934],
       [ 4783, 16032, 11313, ..., 19934, 19934, 19934],
       [ 4471, 11587, 11679, ..., 19934, 19934, 19934]])

In [11]:
trainy

array([[19, 19, 19, ..., 19, 19, 19],
       [ 6,  1,  1, ..., 19, 19, 19],
       [19, 19, 19, ..., 19, 19, 19],
       ...,
       [19, 19, 19, ..., 19, 19, 19],
       [19, 19, 19, ..., 19, 19, 19],
       [19, 19, 19, ..., 19, 19, 19]])

In [12]:
valX, valy = prep_dataset(val_sents, val_tags, sent_tok2idx, tag_tok2idx, vocab)
testX, testy = prep_dataset(test_sents, test_tags, sent_tok2idx, tag_tok2idx, vocab)

In [13]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding, Dropout
from keras.layers import TimeDistributed, Bidirectional



def get_model(vocab_size, embedding_dim, max_len, classes):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, mask_zero=False, input_length=max_len ))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(200, return_sequences=True, recurrent_dropout=0.5)))
    model.add(TimeDistributed(Dropout(0.5)))
    # model.add(Bidirectional(LSTM(16, return_sequences=True, recurrent_dropout=0.2)))
    model.add(TimeDistributed(Dense(classes, activation='softmax')))

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
    return model

In [14]:
model = get_model(len(vocab), 200, 41, len(tag_vocab))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

In [16]:
model.fit(trainX, trainy.reshape(len(trainX),-1,1))

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
5795/5795 [==============================] - 39s 7ms/step - loss: 0.3073 - acc: 0.9616


In [49]:
model.predict_classes(valX)

array([[6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       ...,
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6],
       [6, 6, 6, ..., 6, 6, 6]], dtype=int64)

In [18]:
def evaluate_model(model, dataset, ground_truths, tag_dict):
    preds = model.predict_classes(dataset).tolist()
    flat_preds = [item for sublist in preds for item in sublist]
    flat_truths = [item for sublist in ground_truths for item in sublist]
    flat_preds = [tag_dict[idx] for idx in flat_preds]
    flat_truths = [tag_dict[idx] for idx in flat_truths]
    return classification_report(flat_truths, flat_preds)

In [19]:
model = get_model(len(vocab), 200, 41, len(tag_vocab))
reports, callbacks = [], []
for i in range(10):
    callbacks.append(model.fit(trainX, trainy.reshape(len(trainX),-1,1), validation_data=(valX, valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model(model, valX, valy, tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model(model, trainX, trainy, tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 42s 7ms/step - loss: 0.3054 - acc: 0.9615 - val_loss: 0.1655 - val_acc: 0.9700
Epoch 2/2
5795/5795 [==============================] - 37s 6ms/step - loss: 0.1535 - acc: 0.9683 - val_loss: 0.1340 - val_acc: 0.9707
===========vreport==============
             precision    recall  f1-score   support

        loc       0.12      0.22      0.16       113
     person       0.00      0.00      0.00       112
    company       0.00      0.00      0.00       104
     tvshow       0.00      0.00      0.00         4
      other       0.00      0.00      0.00        81
 sportsteam       0.00      0.00      0.00        20
   facility       0.00      0.00      0.00        34
    product       0.00      0.00      0.00        34
      movie       0.00      0.00      0.00         7
musicartist       0.00      0.00      0.00        28

  micro avg       0.12      0.05      0.07       537
  macro avg   

5795/5795 [==============================] - 35s 6ms/step - loss: 0.0308 - acc: 0.9912 - val_loss: 0.1480 - val_acc: 0.9761
Epoch 2/2
5795/5795 [==============================] - 35s 6ms/step - loss: 0.0253 - acc: 0.9926 - val_loss: 0.1462 - val_acc: 0.9747
===========vreport==============
             precision    recall  f1-score   support

        loc       0.75      0.50      0.60       113
     person       0.44      0.27      0.33       112
    company       0.59      0.53      0.56       104
     tvshow       0.00      0.00      0.00         4
      other       0.25      0.31      0.27        81
 sportsteam       0.25      0.10      0.14        20
   facility       0.47      0.47      0.47        34
    product       0.04      0.03      0.04        34
      movie       0.00      0.00      0.00         7
musicartist       0.03      0.04      0.03        28

  micro avg       0.43      0.35      0.38       537
  macro avg       0.44      0.35      0.38       537

===========train 

In [20]:
print(evaluate_model(model, testX, testy, tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.76      0.55      0.64       165
      other       0.29      0.32      0.30       103
   facility       0.52      0.32      0.39        47
     person       0.71      0.39      0.51       104
    product       0.29      0.14      0.19        28
musicartist       0.07      0.04      0.05        27
    company       0.53      0.39      0.45        84
 sportsteam       0.47      0.26      0.33        31
     tvshow       0.00      0.00      0.00         7
      movie       0.00      0.00      0.00         8

  micro avg       0.51      0.37      0.43       604
  macro avg       0.53      0.37      0.44       604



## BI-LSTM with word embedding with padding mask

In [21]:
def model_with_masking(vocab_size, embedding_dim, max_len, classes):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, mask_zero=True, input_length=max_len ))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(200, return_sequences=True, recurrent_dropout=0.5)))
    model.add(TimeDistributed(Dropout(0.5)))
    # model.add(Bidirectional(LSTM(16, return_sequences=True, recurrent_dropout=0.2)))
    model.add(TimeDistributed(Dense(classes, activation='softmax')))

    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
    return model    

In [22]:
mask_sent_idx2tok, mask_sent_tok2idx, mask_vocab = build_dict(train_sents+val_sents, extra_tokens=['<unk>'], start_idx=1)

In [23]:
mask_sent_idx2tok[0] = '<pad>'
mask_sent_tok2idx['<pad>'] = 0

In [24]:
mask_vocab.add('<pad>')

In [25]:
mask_tag_idx2tok, mask_tag_tok2idx, mask_tag_vocab = build_dict(train_tags+val_tags, start_idx=1)

In [26]:
mask_tag_idx2tok[0] = '<pad>'
mask_tag_tok2idx['<pad>'] = 0
mask_tag_vocab.add('<pad>')

In [27]:
trainX, trainy = prep_dataset(train_sents, train_tags, mask_sent_tok2idx, mask_tag_tok2idx, mask_vocab, mask=True)
valX, valy = prep_dataset(val_sents, val_tags, mask_sent_tok2idx, mask_tag_tok2idx, mask_vocab, mask=True)
testX, testy = prep_dataset(test_sents, test_tags, mask_sent_tok2idx, mask_tag_tok2idx, mask_vocab, mask=True)

In [32]:
trainX

array([[ 7042,  4472, 15734, ...,     0,     0,     0],
       [ 3720,  8143,  1686, ...,     0,     0,     0],
       [19943,  4976,  4472, ...,     0,     0,     0],
       ...,
       [20394,  9186, 12345, ...,     0,     0,     0],
       [ 4784, 16033, 11314, ...,     0,     0,     0],
       [ 4472, 11588, 11680, ...,     0,     0,     0]])

In [33]:
# to evaluate model with padding mask. 
# Removes the unwanted padded tags while evaluation
def evaluate_model_with_padding(model, dataset, ground_truths, tag_dict, padded_value=0):
    preds = model.predict_classes(dataset).tolist() # ground_truths will be list already, so no need to convert
    ground_truths = ground_truths.tolist()
    preds_sliced = []
    ground_truths_sliced = []
    for pred_sent, truth_sent in zip(preds, ground_truths):
        if padded_value in truth_sent:
            index_ = truth_sent.index(padded_value)
            preds_sliced.append(pred_sent[:index_])
            ground_truths_sliced.append(truth_sent[:index_])
        else:
            preds_sliced.append(pred_sent)
            ground_truths_sliced.append(truth_sent)            
        
    flat_preds = [item for sublist in preds_sliced for item in sublist]
    flat_truths = [item for sublist in ground_truths_sliced for item in sublist]
    flat_preds = [tag_dict[idx] for idx in flat_preds]
    flat_truths = [tag_dict[idx] for idx in flat_truths]
    return classification_report(flat_truths, flat_preds) 

In [30]:
model = model_with_masking(len(mask_vocab), 200, 41, len(mask_tag_vocab))
reports, callbacks = [], []
for i in range(10):
    callbacks.append(model.fit(trainX, trainy.reshape(len(trainX),-1,1), validation_data=(valX, valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model_with_padding(model, valX, valy, mask_tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model_with_padding(model, trainX, trainy, mask_tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 43s 7ms/step - loss: 0.6741 - acc: 0.9144 - val_loss: 0.4018 - val_acc: 0.9264
Epoch 2/2
5795/5795 [==============================] - 38s 6ms/step - loss: 0.3592 - acc: 0.9242 - val_loss: 0.3425 - val_acc: 0.9286
===========vreport==============
             precision    recall  f1-score   support

    company       0.00      0.00      0.00       104
      <pad>       0.00      0.00      0.00       724
        loc       0.13      0.19      0.15       113
     person       0.00      0.00      0.00       112
      other       0.00      0.00      0.00        81
   facility       0.00      0.00      0.00        34
    product       0.00      0.00      0.00        34
 sportsteam       0.00      0.00      0.00        20
     tvshow       0.00      0.00      0.00         4
musicartist       0.00      0.00      0.00        28
      movie       0.00      0.00      0.00         7

  micro avg   

             precision    recall  f1-score   support

      <pad>       0.00      0.00      0.00      5794
    company       0.70      0.90      0.79       643
      other       0.63      0.78      0.70       757
    product       0.52      0.73      0.61       318
     person       0.70      0.93      0.80       886
        loc       0.56      0.95      0.70       996
      movie       0.00      0.00      0.00        68
   facility       0.51      0.86      0.64       314
 sportsteam       0.80      0.24      0.37       217
musicartist       0.32      0.39      0.35       232
     tvshow       0.00      0.00      0.00        58

  micro avg       0.60      0.35      0.44     10283
  macro avg       0.26      0.35      0.29     10283

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 39s 7ms/step - loss: 0.0759 - acc: 0.9789 - val_loss: 0.3335 - val_acc: 0.9377
Epoch 2/2
5795/5795 [==============================] - 38s 7ms/step - loss

5795/5795 [==============================] - 39s 7ms/step - loss: 0.0183 - acc: 0.9950 - val_loss: 0.3860 - val_acc: 0.9367
Epoch 2/2
5795/5795 [==============================] - 42s 7ms/step - loss: 0.0154 - acc: 0.9957 - val_loss: 0.3824 - val_acc: 0.9366
===========vreport==============
             precision    recall  f1-score   support

    company       0.63      0.56      0.59       104
      <pad>       0.00      0.00      0.00       724
        loc       0.31      0.51      0.39       113
     person       0.29      0.31      0.30       112
      other       0.11      0.35      0.17        81
   facility       0.45      0.41      0.43        34
    product       0.19      0.15      0.16        34
 sportsteam       0.05      0.20      0.07        20
     tvshow       0.00      0.00      0.00         4
musicartist       0.14      0.18      0.15        28
      movie       0.00      0.00      0.00         7

  micro avg       0.22      0.16      0.19      1261
  macro avg       

In [34]:
print(evaluate_model_with_padding(model, trainX, trainy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.99      1.00      0.99       996
     person       0.99      0.99      0.99       886
    company       0.97      0.99      0.98       643
      other       0.97      0.98      0.97       757
    product       0.98      0.98      0.98       318
 sportsteam       1.00      1.00      1.00       217
   facility       0.96      0.98      0.97       314
musicartist       0.97      0.98      0.98       232
      movie       0.96      0.96      0.96        68
     tvshow       0.88      0.91      0.90        58

  micro avg       0.98      0.99      0.98      4489
  macro avg       0.98      0.99      0.98      4489



In [35]:
print(evaluate_model_with_padding(model, testX, testy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.75      0.55      0.63       165
     tvshow       0.00      0.00      0.00         7
     person       0.45      0.43      0.44       104
      other       0.23      0.33      0.27       103
   facility       0.42      0.34      0.38        47
musicartist       0.13      0.07      0.10        27
    product       0.19      0.11      0.14        28
    company       0.58      0.42      0.49        84
 sportsteam       0.38      0.29      0.33        31
      movie       0.00      0.00      0.00         8

  micro avg       0.44      0.39      0.41       604
  macro avg       0.47      0.39      0.42       604



## BI-LSTM with character level embedding

In [36]:
max_len_word = 0
for sent in train_sents+val_sents:
    sent_max_word = max(len(word) for word in sent)
    max_len_word = max(max_len_word, sent_max_word)

In [37]:
from collections import defaultdict
len_words_dict = defaultdict(int)

for sent in train_sents+val_sents:
    for word in sent:
        len_words_dict[len(word)] = len_words_dict[len(word)]+1

In [38]:
chars = set([w_i for w in mask_vocab for w_i in w])

In [39]:
chars = set(ch_ for ch_ in chars if ord(ch_)<128) # considering only Ascii chars
chars.add('<chr_unk>')

In [40]:
char2idx = {char_:idx for idx, char_ in enumerate(chars,1)}
char2idx['<pad>'] = 0
idx2char = {value:key for key, value in char2idx.items()}

In [41]:
import numpy as np
max_sent_len = 41
max_char_len = 6

sents_container = []
for sent in train_sents:
    sent_container = []
    sent_len = len(sent)
    for word in sent:
        char_container = []
        word_len = len(word)
        if word_len == max_char_len:
            char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word]
        elif word_len < max_char_len:
            char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word]
            char_container = char_container + ([char2idx['<pad>']]*(max_char_len-len(char_container)))
        else:
            char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word[:max_char_len]]
        sent_container.append(char_container)
    if sent_len<max_sent_len:
        padded_tokens = [[char2idx['<pad>']]*max_char_len]*(max_sent_len-sent_len)
        sent_container = sent_container+padded_tokens
        
    sents_container.append(np.array(sent_container))


In [42]:
trainX_char = np.array(sents_container)

In [43]:
def prep_char_dataset(sents, char2idx, chars, max_sent_len, max_char_len):
    """
    prepares dataset needed for character embedding input
    """    
    sents_container = []
    for sent in sents:
        sent_container = []
        sent_len = len(sent)
        for word in sent:
            char_container = []
            word_len = len(word)
            if word_len == max_char_len:
                char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word]
            elif word_len < max_char_len:
                char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word]
                char_container = char_container + ([char2idx['<pad>']]*(max_char_len-len(char_container)))
            else:
                char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word[:max_char_len]]
            sent_container.append(char_container)
        if sent_len<max_sent_len:
            padded_tokens = [[char2idx['<pad>']]*max_char_len]*(max_sent_len-sent_len)
            sent_container = sent_container+padded_tokens

        sents_container.append(np.array(sent_container))
    return np.array(sents_container)

In [44]:
trainX_char = prep_char_dataset(train_sents, char2idx, chars, max_sent_len, max_char_len)
valX_char = prep_char_dataset(val_sents, char2idx, chars, max_sent_len, max_char_len)
testX_char = prep_char_dataset(test_sents, char2idx, chars, max_sent_len, max_char_len)

In [45]:
print(trainX_char.shape)
print(valX_char.shape)
print(testX_char.shape)

(5795, 41, 6)
(724, 41, 6)
(724, 41, 6)


In [46]:
testX_char[1]

array([[33, 70, 64,  0,  0,  0],
       [ 3,  9,  0,  0,  0,  0],
       [ 9,  7, 26, 26, 70, 38],
       [68,  9,  0,  0,  0,  0],
       [75,  0,  0,  0,  0,  0],
       [29, 38,  7, 95, 64,  0],
       [29,  7, 42, 42, 70, 28],
       [30, 81,  0,  0,  0,  0],
       [19,  7, 32, 70,  0,  0],
       [81,  7, 32,  7, 38, 38],
       [30, 81,  0,  0,  0,  0],
       [76,  0,  0,  0,  0,  0],
       [16, 32,  0,  0,  0,  0],
       [75,  0,  0,  0,  0,  0],
       [29,  7, 32, 70,  0,  0],
       [ 9, 41, 16, 16,  7, 38],
       [61,  7, 41, 38,  0,  0],
       [22, 30, 28, 42, 70,  9],
       [83,  0,  0,  0,  0,  0],
       [47, 38, 30, 61, 91, 64],
       [67,  7, 38,  0,  0,  0],
       [61,  7, 41,  0,  0,  0],
       [28, 41, 61,  9,  0,  0],
       [83,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0

In [158]:
testX_char

array([[[ 8, 30, 85,  0,  0,  0],
        [28,  0,  0,  0,  0,  0],
        [13, 30, 51, 16,  0,  0],
        ...,
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0]],

       [[ 8, 16, 85,  0,  0,  0],
        [ 9, 87,  0,  0,  0,  0],
        [87, 88, 82, 82, 16, 20],
        ...,
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0]],

       [[32, 91,  0,  0,  0,  0],
        [74, 42, 43, 32, 27,  0],
        [40,  0,  0,  0,  0,  0],
        ...,
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0]],

       ...,

       [[62, 16, 20, 16,  0,  0],
        [28, 87,  0,  0,  0,  0],
        [88, 46, 20,  0,  0,  0],
        ...,
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0]],

       [[63, 30, 51, 30,  0,  0],
        [21, 20, 16, 30, 82, 13],
        [47, 16, 33, 87,  0,  0],
        .

In [78]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

def char_model(max_len, max_len_char, vocab_size, char_vocab_size, word_embedding_dim, char_embedding_dim, classes):
    
    # converting word to its embedding
    word_in = Input(shape=(max_len,))
    word_embed = Embedding(vocab_size, word_embedding_dim, mask_zero=True)(word_in)
    
    # converting character to its embedding
    char_in = Input(shape=(max_len, max_len_char))
    char_embed = TimeDistributed(Embedding(char_vocab_size, char_embedding_dim, mask_zero=True))(char_in)
    char_lstm = TimeDistributed(LSTM(units=50, return_sequences=False, recurrent_dropout=0.5))(char_embed)
    
    #concatenating the word and character embedding
    embedding_concat = concatenate([word_embed, char_lstm])
    
    # main model
    main_ip = Dropout(0.5)(embedding_concat)
    main_lstm = Bidirectional(LSTM(200, return_sequences=True, recurrent_dropout=0.5))(main_ip)
    drop = Dropout(0.5)(main_lstm)
    out = TimeDistributed(Dense(classes, activation='softmax'))(drop)
    
    model = Model([word_in, char_in], out)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
    return model
    
    

In [59]:
max_len = trainX.shape[-1]
max_len_char = trainX_char.shape[-1]
word_vocab_size = len(mask_vocab)
char_vocab_size = len(chars)+1
word_embed_dim = 200
char_embed_dim = 30
num_classes = len(mask_tag_vocab)

In [79]:
model = char_model(max_len, max_len_char, word_vocab_size, char_vocab_size, word_embed_dim, char_embed_dim, num_classes)

In [65]:
model.fit([trainX, trainX_char], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char], valy.reshape(len(valX),-1,1)), verbose=1, epochs=1)

Train on 5795 samples, validate on 724 samples
Epoch 1/1
5795/5795 [==============================] - 72s 12ms/step - loss: 0.6580 - acc: 0.9148 - val_loss: 0.3852 - val_acc: 0.9265


In [75]:
# to evaluate model with padding mask. 
# Removes the unwanted padded tags while evaluation
# using predict instead of predict classes to work for keras functional models as well
def evaluate_model_with_padding(model, dataset, ground_truths, tag_dict, padded_value=0):
    preds = model.predict(dataset) # ground_truths will be list already, so no need to convert
    preds = np.argmax(preds, axis=-1).tolist()
    ground_truths = ground_truths.tolist()
    preds_sliced = []
    ground_truths_sliced = []
    for pred_sent, truth_sent in zip(preds, ground_truths):
        if padded_value in truth_sent:
            index_ = truth_sent.index(padded_value)
            preds_sliced.append(pred_sent[:index_])
            ground_truths_sliced.append(truth_sent[:index_])
        else:
            preds_sliced.append(pred_sent)
            ground_truths_sliced.append(truth_sent)            
        
    flat_preds = [item for sublist in preds_sliced for item in sublist]
    flat_truths = [item for sublist in ground_truths_sliced for item in sublist]
    flat_preds = [tag_dict[idx] for idx in flat_preds]
    flat_truths = [tag_dict[idx] for idx in flat_truths]
    return classification_report(flat_truths, flat_preds) 

In [80]:
model = char_model(max_len, max_len_char, word_vocab_size, char_vocab_size, word_embed_dim, char_embed_dim, num_classes)
reports, callbacks = [], []
for i in range(15):
    callbacks.append(model.fit([trainX, trainX_char], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char], valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model_with_padding(model, [valX, valX_char], valy, mask_tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model_with_padding(model, [trainX, trainX_char], trainy, mask_tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 55s 10ms/step - loss: 0.6378 - acc: 0.9155 - val_loss: 0.3701 - val_acc: 0.9265
Epoch 2/2
5795/5795 [==============================] - 47s 8ms/step - loss: 0.3305 - acc: 0.9251 - val_loss: 0.3136 - val_acc: 0.9293
===========vreport==============
             precision    recall  f1-score   support

        loc       0.15      0.25      0.19       113
     person       0.00      0.00      0.00       112
     tvshow       0.00      0.00      0.00         4
    company       0.19      0.05      0.08       104
   facility       0.00      0.00      0.00        34
musicartist       0.00      0.00      0.00        28
    product       0.00      0.00      0.00        34
      movie       0.00      0.00      0.00         7
      other       0.02      0.01      0.02        81
 sportsteam       0.00      0.00      0.00        20

  micro avg       0.13      0.06      0.08       537
  macro avg  

5795/5795 [==============================] - 43s 7ms/step - loss: 0.0616 - acc: 0.9820 - val_loss: 0.3235 - val_acc: 0.9365
Epoch 2/2
5795/5795 [==============================] - 44s 8ms/step - loss: 0.0500 - acc: 0.9852 - val_loss: 0.3312 - val_acc: 0.9321
===========vreport==============
             precision    recall  f1-score   support

        loc       0.62      0.47      0.53       113
     person       0.56      0.31      0.40       112
     tvshow       0.00      0.00      0.00         4
    company       0.61      0.55      0.58       104
   facility       0.32      0.41      0.36        34
musicartist       0.05      0.11      0.06        28
    product       0.24      0.15      0.18        34
      movie       0.00      0.00      0.00         7
      other       0.20      0.33      0.25        81
 sportsteam       0.07      0.20      0.11        20

  micro avg       0.35      0.37      0.36       537
  macro avg       0.43      0.37      0.39       537

===========train 

5795/5795 [==============================] - 50s 9ms/step - loss: 0.0125 - acc: 0.9965 - val_loss: 0.4049 - val_acc: 0.9325
Epoch 2/2
5795/5795 [==============================] - 51s 9ms/step - loss: 0.0120 - acc: 0.9964 - val_loss: 0.4132 - val_acc: 0.9344
===========vreport==============
             precision    recall  f1-score   support

        loc       0.67      0.47      0.55       113
     person       0.47      0.32      0.38       112
     tvshow       0.00      0.00      0.00         4
    company       0.63      0.53      0.58       104
   facility       0.45      0.41      0.43        34
musicartist       0.16      0.18      0.17        28
    product       0.14      0.09      0.11        34
      movie       0.00      0.00      0.00         7
      other       0.21      0.36      0.26        81
 sportsteam       0.20      0.25      0.22        20

  micro avg       0.37      0.37      0.37       537
  macro avg       0.45      0.37      0.40       537

===========train 

In [81]:
print(evaluate_model_with_padding(model, [testX, testX_char], testy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.77      0.55      0.64       165
     tvshow       0.00      0.00      0.00         7
     person       0.54      0.39      0.46       104
      other       0.26      0.37      0.31       103
   facility       0.36      0.36      0.36        47
musicartist       0.07      0.11      0.09        27
    product       0.27      0.14      0.19        28
    company       0.64      0.43      0.51        84
 sportsteam       0.23      0.32      0.27        31
      movie       0.11      0.12      0.12         8

  micro avg       0.41      0.40      0.40       604
  macro avg       0.49      0.40      0.43       604



In [82]:
def prep_rev_char_dataset(sents, char2idx, chars, max_sent_len, max_char_len):
    """
    prepares dataset needed for character embedding input considering only last few characters
    """    
    sents_container = []
    for sent in sents:
        sent_container = []
        sent_len = len(sent)
        for word in sent:
            char_container = []
            word_len = len(word)
            if word_len == max_char_len:
                char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word]
            elif word_len > max_char_len:
                char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word[-max_char_len:]]
#                 char_container = char_container + ([char2idx['<pad>']]*(max_char_len-len(char_container)))
            else:
                char_container = [char2idx[ch_] if ch_ in chars else char2idx['<chr_unk>'] for ch_ in word]
                char_container = char_container + ([char2idx['<pad>']]*(max_char_len-len(char_container)))
            sent_container.append(char_container)
        if sent_len<max_sent_len:
            padded_tokens = [[char2idx['<pad>']]*max_char_len]*(max_sent_len-sent_len)
            sent_container = sent_container+padded_tokens

        sents_container.append(np.array(sent_container))
    return np.array(sents_container)

In [85]:
trainX_char_rev = prep_rev_char_dataset(train_sents, char2idx, chars, max_sent_len, 4)
valX_char_rev = prep_rev_char_dataset(val_sents, char2idx, chars, max_sent_len, 4)
testX_char_rev = prep_rev_char_dataset(test_sents, char2idx, chars, max_sent_len, 4)

In [106]:
def char_model_2(max_len, max_len_char, max_len_char_last, vocab_size, char_vocab_size, 
                 word_embedding_dim, char_embedding_dim, classes):
    
    # converting word to its embedding
    word_in = Input(shape=(max_len,))
    word_embed = Embedding(vocab_size, word_embedding_dim, mask_zero=True)(word_in)
    
    # converting character to its embedding
    char_in = Input(shape=(max_len, max_len_char))
    char_embed = TimeDistributed(Embedding(char_vocab_size, char_embedding_dim, mask_zero=True))(char_in)
    char_lstm = TimeDistributed(LSTM(units=30, return_sequences=False, recurrent_dropout=0.5))(char_embed)
    
    # converting the last few characters to its embedding
    char_in2 = Input(shape=(max_len, max_len_char_last))
    char_embed2 = TimeDistributed(Embedding(char_vocab_size, char_embedding_dim, mask_zero=True))(char_in2)
    char_lstm2 = TimeDistributed(LSTM(units=30, return_sequences=False, recurrent_dropout=0.5))(char_embed2)
    
    #concatenating the word and character embedding
    char_embedding_concat = concatenate([word_embed, char_lstm, char_lstm2])
    drop_emb = Dropout(rate=0.3)(char_embedding_concat)
    emb_dense = Dense(300, activation='relu')(drop_emb)
    
    # main model
    main_ip = Dropout(0.5)(emb_dense)
    main_lstm = Bidirectional(LSTM(200, return_sequences=True, recurrent_dropout=0.5))(main_ip)
    drop = Dropout(0.5)(main_lstm)
    out = TimeDistributed(Dense(classes, activation='softmax'))(drop)
    
    model = Model([word_in, char_in, char_in2], out)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])
    return model

In [107]:
model = char_model_2(max_len, max_len_char, max_len_char_last, word_vocab_size, char_vocab_size, word_embed_dim, char_embed_dim, num_classes)

In [111]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           (None, 41, 6)        0                                            
__________________________________________________________________________________________________
input_29 (InputLayer)           (None, 41, 4)        0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
time_distributed_48 (TimeDistri (None, 41, 6, 30)    2880        input_28[0][0]                   
___________________________________________________________________________________________

In [112]:
max_len = trainX.shape[-1]
max_len_char = trainX_char.shape[-1]
word_vocab_size = len(mask_vocab)
char_vocab_size = len(chars)+1
word_embed_dim = 200
char_embed_dim = 30
num_classes = len(mask_tag_vocab)
max_len_char_last = 4

model = char_model_2(max_len, max_len_char, max_len_char_last, word_vocab_size, char_vocab_size, word_embed_dim, char_embed_dim, num_classes)
reports, callbacks = [], []
for i in range(15):
    callbacks.append(model.fit([trainX, trainX_char, trainX_char_rev], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char, valX_char_rev], valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model_with_padding(model, [valX, valX_char, valX_char_rev], valy, mask_tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model_with_padding(model, [trainX, trainX_char, trainX_char_rev], trainy, mask_tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 64s 11ms/step - loss: 0.5643 - acc: 0.9193 - val_loss: 0.3421 - val_acc: 0.9266
Epoch 2/2
5795/5795 [==============================] - 52s 9ms/step - loss: 0.2974 - acc: 0.9260 - val_loss: 0.3071 - val_acc: 0.9296
===========vreport==============
             precision    recall  f1-score   support

        loc       0.19      0.23      0.21       113
     person       0.11      0.07      0.09       112
     tvshow       0.00      0.00      0.00         4
    company       0.14      0.09      0.11       104
   facility       0.00      0.00      0.00        34
musicartist       0.00      0.00      0.00        28
    product       0.00      0.00      0.00        34
      movie       0.00      0.00      0.00         7
      other       0.00      0.00      0.00        81
 sportsteam       0.00      0.00      0.00        20

  micro avg       0.14      0.08      0.10       537
  macro avg  

5795/5795 [==============================] - 51s 9ms/step - loss: 0.0577 - acc: 0.9832 - val_loss: 0.3522 - val_acc: 0.9345
Epoch 2/2
5795/5795 [==============================] - 52s 9ms/step - loss: 0.0497 - acc: 0.9854 - val_loss: 0.3486 - val_acc: 0.9331
===========vreport==============
             precision    recall  f1-score   support

        loc       0.61      0.44      0.51       113
     person       0.46      0.24      0.32       112
     tvshow       0.00      0.00      0.00         4
    company       0.69      0.53      0.60       104
   facility       0.34      0.35      0.35        34
musicartist       0.08      0.21      0.12        28
    product       0.11      0.12      0.11        34
      movie       0.00      0.00      0.00         7
      other       0.25      0.31      0.28        81
 sportsteam       0.19      0.25      0.22        20

  micro avg       0.34      0.34      0.34       537
  macro avg       0.43      0.34      0.37       537

===========train 

5795/5795 [==============================] - 52s 9ms/step - loss: 0.0175 - acc: 0.9948 - val_loss: 0.4387 - val_acc: 0.9253
Epoch 2/2
5795/5795 [==============================] - 52s 9ms/step - loss: 0.0170 - acc: 0.9949 - val_loss: 0.4252 - val_acc: 0.9293
===========vreport==============
             precision    recall  f1-score   support

        loc       0.63      0.52      0.57       113
     person       0.41      0.27      0.32       112
     tvshow       0.00      0.00      0.00         4
    company       0.65      0.55      0.59       104
   facility       0.34      0.44      0.38        34
musicartist       0.09      0.21      0.13        28
    product       0.11      0.12      0.11        34
      movie       0.00      0.00      0.00         7
      other       0.28      0.33      0.30        81
 sportsteam       0.12      0.25      0.16        20

  micro avg       0.33      0.38      0.35       537
  macro avg       0.42      0.38      0.39       537

===========train 

In [113]:
for i in range(5):
    callbacks.append(model.fit([trainX, trainX_char, trainX_char_rev], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char, valX_char_rev], valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model_with_padding(model, [valX, valX_char, valX_char_rev], valy, mask_tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model_with_padding(model, [trainX, trainX_char, trainX_char_rev], trainy, mask_tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 50s 9ms/step - loss: 0.0097 - acc: 0.9970 - val_loss: 0.4633 - val_acc: 0.9291
Epoch 2/2
5795/5795 [==============================] - 52s 9ms/step - loss: 0.0099 - acc: 0.9970 - val_loss: 0.4688 - val_acc: 0.9277
===========vreport==============
             precision    recall  f1-score   support

        loc       0.71      0.53      0.61       113
     person       0.41      0.31      0.35       112
     tvshow       0.00      0.00      0.00         4
    company       0.62      0.53      0.57       104
   facility       0.33      0.50      0.40        34
musicartist       0.09      0.21      0.13        28
    product       0.06      0.06      0.06        34
      movie       0.00      0.00      0.00         7
      other       0.28      0.31      0.30        81
 sportsteam       0.09      0.25      0.13        20

  micro avg       0.33      0.38      0.36       537
  macro avg   

In [114]:
print(evaluate_model_with_padding(model, [testX, testX_char, testX_char_rev], testy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.75      0.53      0.62       165
     tvshow       0.00      0.00      0.00         7
     person       0.43      0.45      0.44       104
      other       0.32      0.36      0.34       103
   facility       0.55      0.45      0.49        47
musicartist       0.07      0.04      0.05        27
    product       0.17      0.14      0.15        28
    company       0.55      0.46      0.50        84
 sportsteam       0.18      0.32      0.23        31
      movie       0.11      0.25      0.15         8

  micro avg       0.42      0.41      0.41       604
  macro avg       0.47      0.41      0.43       604



In [118]:
# modifying to remove tokens with less count
def build_dict(sents, extra_tokens= None, start_idx=0, min_word_count=1):
    counter = defaultdict(int)
    for sent in sents:
        for word in sent:
            counter[word]+=1
            
    vocab = set()
    tok2idx = {}
    idx2tok = {}
    for key, value in counter.items():
        if value >= min_word_count:
            vocab.add(key)
            
    if extra_tokens:
        vocab.update(extra_tokens)
    
    for i, tok in enumerate(vocab, start_idx):
        idx2tok[i] = tok
    for key, val in idx2tok.items():
        tok2idx[val] = key
    return idx2tok, tok2idx, vocab

In [119]:
mask_sent_idx2tok, mask_sent_tok2idx, mask_vocab = build_dict(train_sents+val_sents, extra_tokens=['<unk>'], start_idx=1, min_word_count=2)

In [122]:
mask_sent_idx2tok[0] = '<pad>'
mask_sent_tok2idx['<pad>'] = 0

In [123]:
trainX, trainy = prep_dataset(train_sents, train_tags, mask_sent_tok2idx, mask_tag_tok2idx, mask_vocab, mask=True)
valX, valy = prep_dataset(val_sents, val_tags, mask_sent_tok2idx, mask_tag_tok2idx, mask_vocab, mask=True)
testX, testy = prep_dataset(test_sents, test_tags, mask_sent_tok2idx, mask_tag_tok2idx, mask_vocab, mask=True)

In [127]:
max_len = trainX.shape[-1]
max_len_char = trainX_char.shape[-1]
word_vocab_size = len(mask_vocab)+1 # forgot to add padding token in mask vocab
char_vocab_size = len(chars)+1
word_embed_dim = 200
char_embed_dim = 30
num_classes = len(mask_tag_vocab)
max_len_char_last = 4

model = char_model_2(max_len, max_len_char, max_len_char_last, word_vocab_size, char_vocab_size, word_embed_dim, char_embed_dim, num_classes)
reports, callbacks = [], []
for i in range(15):
    callbacks.append(model.fit([trainX, trainX_char, trainX_char_rev], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char, valX_char_rev], valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model_with_padding(model, [valX, valX_char, valX_char_rev], valy, mask_tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model_with_padding(model, [trainX, trainX_char, trainX_char_rev], trainy, mask_tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 57s 10ms/step - loss: 0.5582 - acc: 0.9189 - val_loss: 0.3617 - val_acc: 0.9265
Epoch 2/2
5795/5795 [==============================] - 44s 8ms/step - loss: 0.3264 - acc: 0.9246 - val_loss: 0.2905 - val_acc: 0.9288
===========vreport==============
             precision    recall  f1-score   support

        loc       0.12      0.14      0.13       113
     person       0.00      0.00      0.00       112
     tvshow       0.00      0.00      0.00         4
    company       0.08      0.03      0.04       104
   facility       0.00      0.00      0.00        34
musicartist       0.00      0.00      0.00        28
    product       0.00      0.00      0.00        34
      movie       0.00      0.00      0.00         7
      other       0.02      0.01      0.01        81
 sportsteam       0.00      0.00      0.00        20

  micro avg       0.08      0.04      0.05       537
  macro avg  

5795/5795 [==============================] - 45s 8ms/step - loss: 0.1402 - acc: 0.9612 - val_loss: 0.2574 - val_acc: 0.9448
Epoch 2/2
5795/5795 [==============================] - 45s 8ms/step - loss: 0.1295 - acc: 0.9630 - val_loss: 0.2783 - val_acc: 0.9466
===========vreport==============
             precision    recall  f1-score   support

        loc       0.67      0.50      0.57       113
     person       0.46      0.33      0.39       112
     tvshow       0.00      0.00      0.00         4
    company       0.68      0.52      0.59       104
   facility       0.45      0.38      0.41        34
musicartist       0.44      0.14      0.22        28
    product       0.22      0.12      0.15        34
      movie       0.00      0.00      0.00         7
      other       0.31      0.28      0.30        81
 sportsteam       0.20      0.05      0.08        20

  micro avg       0.51      0.36      0.42       537
  macro avg       0.49      0.36      0.41       537

===========train 

5795/5795 [==============================] - 44s 8ms/step - loss: 0.0731 - acc: 0.9773 - val_loss: 0.3339 - val_acc: 0.9445
Epoch 2/2
5795/5795 [==============================] - 44s 8ms/step - loss: 0.0702 - acc: 0.9786 - val_loss: 0.3375 - val_acc: 0.9451
===========vreport==============
             precision    recall  f1-score   support

        loc       0.57      0.56      0.56       113
     person       0.60      0.44      0.51       112
     tvshow       0.00      0.00      0.00         4
    company       0.60      0.54      0.57       104
   facility       0.52      0.47      0.49        34
musicartist       0.38      0.18      0.24        28
    product       0.29      0.15      0.20        34
      movie       0.00      0.00      0.00         7
      other       0.31      0.32      0.31        81
 sportsteam       0.21      0.15      0.18        20

  micro avg       0.49      0.42      0.45       537
  macro avg       0.49      0.42      0.44       537

===========train 

In [128]:
for i in range(5):
    callbacks.append(model.fit([trainX, trainX_char, trainX_char_rev], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char, valX_char_rev], valy.reshape(len(valX),-1,1)), verbose=1, epochs=2))
#     val_preds_ = model.predict_classes(valX)
    print("===========vreport==============")
    vreport = evaluate_model_with_padding(model, [valX, valX_char, valX_char_rev], valy, mask_tag_idx2tok)
    print(vreport)
    print("===========train report==============")
    tr_report = evaluate_model_with_padding(model, [trainX, trainX_char, trainX_char_rev], trainy, mask_tag_idx2tok)
    print(tr_report)
    reports.append((vreport, tr_report))

Train on 5795 samples, validate on 724 samples
Epoch 1/2
5795/5795 [==============================] - 43s 7ms/step - loss: 0.0479 - acc: 0.9847 - val_loss: 0.3880 - val_acc: 0.9449
Epoch 2/2
5795/5795 [==============================] - 46s 8ms/step - loss: 0.0471 - acc: 0.9847 - val_loss: 0.3853 - val_acc: 0.9418
===========vreport==============
             precision    recall  f1-score   support

        loc       0.58      0.57      0.57       113
     person       0.54      0.44      0.48       112
     tvshow       0.00      0.00      0.00         4
    company       0.59      0.55      0.57       104
   facility       0.45      0.44      0.45        34
musicartist       0.22      0.21      0.22        28
    product       0.24      0.18      0.20        34
      movie       0.00      0.00      0.00         7
      other       0.26      0.32      0.29        81
 sportsteam       0.20      0.20      0.20        20

  micro avg       0.44      0.42      0.43       537
  macro avg   

In [129]:
print(evaluate_model_with_padding(model, [testX, testX_char, testX_char_rev], testy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.67      0.58      0.62       165
     tvshow       0.00      0.00      0.00         7
     person       0.45      0.57      0.50       104
      other       0.30      0.35      0.32       103
   facility       0.47      0.40      0.44        47
musicartist       0.30      0.11      0.16        27
    product       0.10      0.07      0.08        28
    company       0.54      0.39      0.46        84
 sportsteam       0.30      0.23      0.26        31
      movie       0.00      0.00      0.00         8

  micro avg       0.46      0.42      0.44       604
  macro avg       0.46      0.42      0.43       604



In [130]:
model = char_model_2(max_len, max_len_char, max_len_char_last, word_vocab_size, char_vocab_size, word_embed_dim, char_embed_dim, num_classes)
callback_ = model.fit([trainX, trainX_char, trainX_char_rev], trainy.reshape(len(trainX),-1,1), validation_data=([valX, valX_char, valX_char_rev], valy.reshape(len(valX),-1,1)),batch_size=16, verbose=1, epochs=50)

Train on 5795 samples, validate on 724 samples
Epoch 1/50
5795/5795 [==============================] - 82s 14ms/step - loss: 0.4707 - acc: 0.9215 - val_loss: 0.3154 - val_acc: 0.9289
Epoch 2/50
5795/5795 [==============================] - 69s 12ms/step - loss: 0.2933 - acc: 0.9287 - val_loss: 0.2680 - val_acc: 0.9328
Epoch 3/50
5795/5795 [==============================] - 63s 11ms/step - loss: 0.2475 - acc: 0.9368 - val_loss: 0.2487 - val_acc: 0.9379
Epoch 4/50
5795/5795 [==============================] - 63s 11ms/step - loss: 0.2169 - acc: 0.9441 - val_loss: 0.2429 - val_acc: 0.9446
Epoch 5/50
5795/5795 [==============================] - 63s 11ms/step - loss: 0.1950 - acc: 0.9490 - val_loss: 0.2397 - val_acc: 0.9445
Epoch 6/50
5795/5795 [==============================] - 70s 12ms/step - loss: 0.1794 - acc: 0.9530 - val_loss: 0.2390 - val_acc: 0.9467
Epoch 7/50
5795/5795 [==============================] - 72s 12ms/step - loss: 0.1640 - acc: 0.9558 - val_loss: 0.2413 - val_acc: 0.9445
E

In [131]:
print(evaluate_model_with_padding(model, [testX, testX_char, testX_char_rev], testy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.71      0.60      0.65       165
     tvshow       0.00      0.00      0.00         7
     person       0.43      0.55      0.48       104
      other       0.30      0.39      0.34       103
   facility       0.50      0.47      0.48        47
musicartist       0.22      0.15      0.18        27
    product       0.09      0.11      0.10        28
    company       0.51      0.43      0.46        84
 sportsteam       0.39      0.29      0.33        31
      movie       0.00      0.00      0.00         8

  micro avg       0.45      0.45      0.45       604
  macro avg       0.46      0.45      0.45       604



In [133]:
print(evaluate_model_with_padding(model, [trainX, trainX_char, trainX_char_rev], trainy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.99      0.98      0.99       996
     person       0.99      0.99      0.99       886
    company       0.97      0.97      0.97       643
      other       0.95      0.97      0.96       757
    product       0.93      0.95      0.94       318
 sportsteam       0.99      0.99      0.99       217
   facility       0.96      0.98      0.97       314
musicartist       0.92      0.97      0.94       232
      movie       0.97      0.99      0.98        68
     tvshow       0.95      0.97      0.96        58

  micro avg       0.97      0.98      0.97      4489
  macro avg       0.97      0.98      0.97      4489



In [134]:
print(evaluate_model_with_padding(model, [valX, valX_char, valX_char_rev], valy, mask_tag_idx2tok))

             precision    recall  f1-score   support

        loc       0.67      0.58      0.62       113
     person       0.57      0.48      0.52       112
     tvshow       0.00      0.00      0.00         4
    company       0.62      0.58      0.60       104
   facility       0.55      0.50      0.52        34
musicartist       0.19      0.18      0.19        28
    product       0.19      0.21      0.20        34
      movie       0.00      0.00      0.00         7
      other       0.29      0.38      0.33        81
 sportsteam       0.17      0.25      0.20        20

  micro avg       0.46      0.46      0.46       537
  macro avg       0.49      0.46      0.47       537



### Gained a performance boost of around 3-5 percent when switched to charcter embedding

In [135]:
preds = model.predict([valX, valX_char, valX_char_rev])

In [136]:
preds.shape

(724, 41, 22)

In [138]:
np.argmax(preds, axis=-1).shape

(724, 41)

In [139]:
mask_tag_idx2tok[20]

'O'

In [142]:
def predict_tags(model, data, tag2word):
    preds = model.predict(data)
    preds = np.argmax(preds, axis=-1).tolist()
    out_container=[]
    for sent in preds:
        out_container.append([tag2word[tag] for tag in sent])
    return out_container

In [143]:
out_tags = predict_tags(model, [valX, valX_char, valX_char_rev], mask_tag_idx2tok)

In [155]:
display_number_results = 5
from_value = 528
for i in range(from_value, from_value+display_number_results):
    for out, val in zip(out_tags[i], val_tags[i]):
        print(f'predicted-->{out}           actual-->{val}')
    print('='*10)

predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->B-geo-loc           actual-->B-geo-loc
predicted-->I-geo-loc           actual-->I-geo-loc
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->O           actual-->O
predicted-->B-musicartist           actual-->B-musicartist
predicted-->O           actual-->O
predicted-->O           actual-->O